In [15]:
from collections import defaultdict
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

def load_data(path):
    data = []
    cols = {'verified':0,'reviewTime':1,'reviewerID':2,'asin':3,"reviewText":4,"summary":5,"unixReviewTime":6,"sentiment":7,"id":8}
    for line in open(path):
        review_data = json.loads(line)
        tmp = [None]*len(cols)
        for key in review_data:
            if key in cols:
                if key == "sentiment":
                    tmp[cols[key]] = 1 if review_data[key] == "positive" else 0
                else:
                    tmp[cols[key]] = str(review_data[key])
        data.append(tmp)
    X = pd.DataFrame(data, columns=cols)
    # set empty reviews to '' (instead of None)
    X.loc[X['reviewText'].isna(), 'reviewText'] = ''
    X.loc[X['summary'].isna(), 'summary'] = ''
    y = X['sentiment']
    X.drop(columns='sentiment', inplace=True)
    return X, y

df, target = load_data('Data/music_reviews_train.json')
df_dev, y_dev = load_data('Data/music_reviews_dev.json')
df_test, y_test = load_data('Data/music_reviews_test_masked.json')
df_hard, y_hard = load_data('Data/phase_2_masked.json')
sum(target)/len(target)

0.60783

In [76]:
from random import sample
import re
ranges = [10, 50, 100, 500, 2000]

samples = df.reviewText[df.reviewText.str.contains(r'[a-zA-Z]')]
for r in ranges:
    idxs = sample(range(len(samples)), r)
    text = samples.iloc[idxs].str.replace('\n', ' ')
    label = target[idxs]
    
    with open(f'Data/subsets/n_{r}.txt', 'w') as f:
        for l, t in zip(label, text):
            f.write(f'{l}\t{t}\n')